In [21]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [22]:
HF_TOKEN = os.environ.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN

In [23]:
import langchain
from langchain.document_loaders import PyPDFLoader

In [24]:
data_harry_potter = PyPDFLoader(r'E:\Generative_AI_Basics_to_Advance\ADVANCE_RAG\PART-05\Data\harrypotter-50-70.pdf')

In [25]:
doc_harry_potter = data_harry_potter.load()

In [26]:
len(doc_harry_potter)

21

In [27]:
data_got = PyPDFLoader(r'E:\Generative_AI_Basics_to_Advance\ADVANCE_RAG\PART-05\Data\got_pdf-10-30.pdf')

In [28]:
doc_got = data_got.load()

In [29]:
len(doc_got)

21

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

In [31]:
text_harrypotter = text_splitter.split_documents(doc_harry_potter)

In [32]:
text_got = text_splitter.split_documents(doc_got)

In [33]:
from langchain.embeddings import HuggingFaceEmbeddings,HuggingFaceBgeEmbeddings

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [34]:
from langchain.vectorstores import Chroma
harrypotter_vectorstore = Chroma.from_documents(text_harrypotter,
                                       hf_embeddings)

In [35]:
text_got_vectorstore = Chroma.from_documents(text_got,hf_embeddings)

In [62]:
retriever_harrypotter = harrypotter_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})

In [64]:
retriever_harrypotter.invoke("who is harry potter?")

[Document(metadata={'author': 'Rowling, J.K.', 'creationdate': '2019-02-21T16:04:23+00:00', 'creator': 'calibre 3.27.1 [https://calibre-ebook.com]', 'page': 356, 'page_label': '357', 'producer': 'calibre 3.27.1 [https://calibre-ebook.com]', 'source': 'E:\\Generative_AI_Basics_to_Advance\\ADVANCE_RAG\\PART-05\\Data\\harrypotter.pdf', 'title': 'Harry Potter: The Complete Collection', 'total_pages': 3623}, page_content='him,\the’s\tan\tinternationally\tfamous\twizard\talready!’\tBut\twhen\tI\twas\ttwelve,\tI\nwas\tjust\tas\tmuch\tof\ta\tnobody\tas\tyou\tare\tnow.\tIn\tfact,\tI’d\tsay\tI\twas\teven\tmore\nof\ta\tnobody!\tI\tmean,\ta\tfew\tpeople\thave\theard\tof\tyou,\thaven’t\tthey?\tAll\tthat\nbusiness\twith\tHe-Who-Must-Not-Be-Named!”\tHe\tglanced\tat\tthe\tlightning\nscar\ton\tHarry’s\tforehead.\t“I\tknow,\tI\tknow\t—\tit’s\tnot\tquite\tas\tgood\tas\twinning\nWitch\tWeekly\n’s\tMost-Charming-Smile\tAward\tfive\ttimes\tin\ta\trow,\tas\tI\thave\t—\nbut\tit’s\ta\t\nstart,\n\tHarry,\tit’s\

In [65]:

retriever_got = text_got_vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})

In [66]:
retriever_got.invoke("who is jon?")

[Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-02-20T17:20:43+00:00', 'page': 7, 'page_label': '8', 'producer': 'iLovePDF', 'source': 'E:\\Generative_AI_Basics_to_Advance\\ADVANCE_RAG\\PART-05\\Data\\got_pdf-10-30.pdf', 'total_pages': 21}, page_content='Riverrun. “He had courage, at the least.”\n“No,” Jon Snow said quietly. “It was not courage. This one was dead of fear. You could \nsee it in his eyes, Stark.” Jon’s eyes were a grey so dark they seemed almost black, but'),
 Document(metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-02-20T17:20:43+00:00', 'page': 18, 'page_label': '19', 'producer': 'iLovePDF', 'source': 'E:\\Generative_AI_Basics_to_Advance\\ADVANCE_RAG\\PART-05\\Data\\got_pdf-10-30.pdf', 'total_pages': 21}, page_content='“It was the king’s seal, and the letter is in Robert’s own hand. I saved it for you. He said \nLord Arryn was taken quickly. Even Maester Pycelle was helpless, but he brought the \nmilk of the poppy, so Jon

In [67]:
from langchain.retrievers import MergerRetriever

merged_retriever = MergerRetriever(retrievers=[retriever_harrypotter, retriever_got])

In [70]:
for chunks in merged_retriever.get_relevant_documents("Who was the jon snow?"):
    print(chunks.page_content)

Riverrun. “He had courage, at the least.”
“No,” Jon Snow said quietly. “It was not courage. This one was dead of fear. You could 
see it in his eyes, Stark.” Jon’s eyes were a grey so dark they seemed almost black, but
Riverrun. “He had courage, at the least.”
“No,” Jon Snow said quietly. “It was not courage. This one was dead of fear. You could 
see it in his eyes, Stark.” Jon’s eyes were a grey so dark they seemed almost black, but
Bran saw his father’s face change, saw the other men exchange glances. He loved Jon 
with all his heart at that moment. Even at seven, Bran understood what his brother had 
done. The count had come right only because Jon had omitted himself. He had included 
the girls, included even Rickon, the baby, but not the bastard who bore the surname 
Snow, the name that custom decreed be given to all those in the north unlucky enough to 
be born with no name of their own.
Bran saw his father’s face change, saw the other men exchange glances. He loved Jon 
with all 

C:\Users\parth\AppData\Local\Temp\ipykernel_21472\37220557.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  for chunks in merged_retriever.get_relevant_documents("Who was the jon snow?"):


In [72]:
for chunks in merged_retriever.get_relevant_documents("Who is a harry potter?"):
    print(chunks.page_content)

him,	he’s	an	internationally	famous	wizard	already!’	But	when	I	was	twelve,	I
was	just	as	much	of	a	nobody	as	you	are	now.	In	fact,	I’d	say	I	was	even	more
of	a	nobody!	I	mean,	a	few	people	have	heard	of	you,	haven’t	they?	All	that
business	with	He-Who-Must-Not-Be-Named!”	He	glanced	at	the	lightning
scar	on	Harry’s	forehead.	“I	know,	I	know	—	it’s	not	quite	as	good	as	winning
Witch	Weekly
’s	Most-Charming-Smile	Award	five	times	in	a	row,	as	I	have	—
but	it’s	a	
start,
	Harry,	it’s	a	
start.
”
him,	he’s	an	internationally	famous	wizard	already!’	But	when	I	was	twelve,	I
was	just	as	much	of	a	nobody	as	you	are	now.	In	fact,	I’d	say	I	was	even	more
of	a	nobody!	I	mean,	a	few	people	have	heard	of	you,	haven’t	they?	All	that
business	with	He-Who-Must-Not-Be-Named!”	He	glanced	at	the	lightning
scar	on	Harry’s	forehead.	“I	know,	I	know	—	it’s	not	quite	as	good	as	winning
Witch	Weekly
’s	Most-Charming-Smile	Award	five	times	in	a	row,	as	I	have	—
but	it’s	a	
start,
	Harry,	it’s	a	
start.
”
Text

In [73]:
from langchain.document_transformers import EmbeddingsClusteringFilter,EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder

In [74]:

from re import search
filter = EmbeddingsRedundantFilter(embeddings=hf_embeddings)
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=merged_retriever,search_kwargs={"k": 3, "include_metadata": True}
)

In [75]:
docs = compression_retriever_reordered.get_relevant_documents("What is esops?")
print(len(docs))
print(docs[0].page_content)

5
took	a	large	swig	of	pumpkin	juice.	Then	he	said,	“Listen	.	.	.	you	don’t	fancy
going	out	a	bit	earlier	with	me,	do	you?	Just	to	—	er	—	give	me	some
practice	before	training?	So	I	can,	you	know,	get	my	eye	in	a	bit	.	.	.”


In [1]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
import os
HF_TOKEN = os.environ.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN
model = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    huggingfacehub_api_token=HF_TOKEN
)
llm = ChatHuggingFace(llm=model)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
llm.invoke("tell me the Theorem of Pithagorus?")

AIMessage(content=' The Theorem of Pythagorus is a fundamental result in geometry, named after the ancient Greek mathematician Pythagoras. It relates to the sides of a right-angled triangle and states that:\n\nIn a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides.\n\nMathematically, if a and b are the lengths of the two legs (the sides adjacent to the right angle), and c is the length of the hypotenuse, then:\n\na² + b² = c²\n\nThis theorem has wide applications in various fields of mathematics and physics. It can be used to find the length of the hypotenuse (or any of the sides) if the lengths of the other two sides are known.', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=190, prompt_tokens=18, total_tokens=208), 'model': '', 'finish_reason': 'stop'}, id='run-54b77e40-c881-46a8-a52b-f74a10b95e1d-0')

In [3]:
result = llm.invoke("how to prove Pythagorus theorem?in detailed way")

In [4]:
from IPython.display import Markdown

display(Markdown(result.content))

 The Pythagorean Theorem is a fundamental relationship between the sides of a right-angled triangle. It states that the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. Mathematically, this can be expressed as:

a² + b² = c²

where a and b are the lengths of the legs (the two sides adjacent to the right angle), and c is the length of the hypotenuse.

There are several ways to prove the Pythagorean Theorem, but here is a detailed andgeometric proof:

1. Start with a right-angled triangle ABC, where the hypotenuse is side BC.
2. Draw a square on each side of the triangle. The square on side a has side length a, so it contains an area of a² lattice units. Similarly, the square on side b has side length b and contains an area of b².
3. Place the right-angled triangle ABC inside the squares such that side a is the base of the triangle and side b is the height. Since the triangle is right-angled, we know that the area of the triangle is given by the product of half the base and the height (i.e., 1/2 * a * b).
4. Now, construct a rectangle DEFG, where DE = BC, DF = AB, and EF = a. Since DE and BC are equal, we can place triangle ABC inside square DEFG such that its base rests on DE and its vertex C lies on diagonal FG.
5. The area of rectangle DEFG is given by the product of its length and width; that is, DE * EF = BC * a.
6. However, we know that the area of square DE is BC², and the area of square EF is a². Therefore, the area of square DE plus the area of square EF is BC² + a².
7. The total area of the larger rectangle DEFG is the sum of the areas of the right-angled triangle ABC and the two squares. That is, Area(DEFG) = Area(ABC) + Area(square on side a) + Area(square on side b).
8. Using the formula for the area of a right-angled triangle (1/2 * base * height), we have Area(ABC) = 1/2 * a * b. Also, from step 6, we know that Area(DEFG) = BC² + a².
9. Equating the areas, we get 1/2 * a * b + a² + b² = BC²+ a².
10. Expanding BC² = (BC)², we get 1/2 * a * b + a² + b² = (BC)² + a².
11. Since the side lengths a, b, and c are given, we can set BC = c. Thus, 1/2 * a * b + a² + b² = c² + a².
12. Rearranging the terms, we have a²+ b² = c², which is the Pythagorean Theorem.

Therefore, by constructing the squares and rectangles, and equating their areas, we have successfully proved the Pythagorean Theorem.

In [ ]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever = compression_retriever_reordered,
      return_source_documents = True
)

In [80]:
qa("who is jon")['result']

' Jon Snow is a character in the "A Song of Ice and Fire" series by George R.R. Martin. He is a member of the Night\'s Watch and has been mentioned in the provided context as having found something important that he wished his father, Eddard Stark, to see. Eddard Stark is the Lord of Winterfell and Warden of the North, and he is also referred to as Lord Stark in the text. Bran Stark, another character in the series, is the one who narrates the context and relays the conversation between Eddard Stark and Jon Snow. The context indicates that Jon Snow is someone who is beloved by the Stark family and has a deep connection to them. The specific details about Jon\'s identity, such as his age, appearance, or background, are not provided in the given context.'

In [81]:
qa("who is harry potter?")['result']

' Harry Potter is a famous wizard who attended Hogwarts School of Witchcraft and Wizardry. He became famous for defeating the dark wizard Voldemort at a young age. Before that, Harry was an ordinary boy, despite some recognition for his scar and his modest demeanor.'

In [82]:
qa('What is esops?')['result']

" Esop is not mentioned in the given context. It seems like there's a misunderstanding or a typo in the user's question. Esop could potentially refer to Aesop's Fables, but there's no connection to the text provided. If you meant to ask about something else, please provide more context or clarify your question."

In [83]:
qa("what is provided in the given document?")

{'query': 'what is provided in the given document?',
 'result': " The document provides a list of requirements for a first-year student's uniform at Hogwarts School of Witchcraft and Wizardry. The list includes three sets of plain work robes (black), one plain pointed hat (black) for day wear, one pair of protective gloves (dragon hide or similar), and one winter cloak (black, silver fastenings).",
 'source_documents': [_DocumentWithState(metadata={'author': 'Rowling, J.K.', 'creationdate': '2019-02-21T16:04:23+00:00', 'creator': 'calibre 3.27.1 [https://calibre-ebook.com]', 'page': 521, 'page_label': '522', 'producer': 'calibre 3.27.1 [https://calibre-ebook.com]', 'source': 'E:\\Generative_AI_Basics_to_Advance\\ADVANCE_RAG\\PART-05\\Data\\harrypotter.pdf', 'title': 'Harry Potter: The Complete Collection', 'total_pages': 3623}, page_content='paper\tthat\tHarry\twas\tsure\the\twas\tgoing\tto\ttear\tit.\tWhile\tRon\tkept\twatch\the\ntugged\tand\ttwisted,\tand\tat\tlast,\tafter\tseveral\t